In [2]:
# Install the required packages
%pip install datasets pandas
from datasets import load_dataset
import re
import psutil
import sys
import time
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


/home/tam/miniconda3/envs/logosdb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the dataset
ds = load_dataset("jordiclive/wikipedia-summary-dataset")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
# Preprocess to remove all empty, special characters and convert to lowercase
# Helper function to clean the text by removing special characters and converting to lowercase
def clean_text(example):
    for key, value in example.items():
        if isinstance(value, str):
            value = value.replace('\n', ' ').replace('\t', ' ')  # Replace newlines with spaces
            value = re.sub(r'[^a-zA-Z0-9\s]', '', value)  # Remove special characters
            example[key] = value.lower()  # Convert to lowercase
    return example
    
def preprocess_data(ds):
    # Print the number of rows before filtering
    print(f"Rows before filtering: {ds.num_rows}")

    # Initialize a list to store the first 5 rows with missing values
    rows_with_missing_values = []

    # Iterate through the dataset and collect the first 5 rows with missing values
    for row in ds:
        if any(value is None or value != value for value in row.values()):
            rows_with_missing_values.append(row)
        if len(rows_with_missing_values) >= 5:
            break

    # Convert the subset to a pandas DataFrame and print the first 5 rows
    sub_df_pd = pd.DataFrame(rows_with_missing_values)
    print("First 5 rows with missing values:")
    print(sub_df_pd)
    
    # Remove rows with any None or NaN values
    ds = ds.filter(lambda x: all(value is not None and value == value for value in x.values()))
    
    # Print the number of rows after filtering
    print(f"Rows after filtering: {ds.num_rows}")
    
    # Remove special characters and convert to lowercase
    ds = ds.map(clean_text)

    # Convert to pandas DataFrame
    return ds.to_pandas()


In [5]:
def preprocess_data(ds):
    # Print the number of rows before filtering
    print(f"Rows before filtering: {ds.num_rows}")
    
    # Initialize a list to store the first 5 rows with missing values
    rows_with_missing_values = []
    
    # Iterate through the dataset and collect the first 5 rows with missing values
    for row in ds:
        if any(value is None or value != value for value in row.values()):
            rows_with_missing_values.append(row)
        if len(rows_with_missing_values) >= 5:
            break
    
    # Convert the subset to a pandas DataFrame and print the first 5 rows
    sub_df_pd = pd.DataFrame(rows_with_missing_values)
    print("First 5 rows with missing values:")
    print(sub_df_pd)
    
    # Remove rows with any None or NaN values
    ds = ds.filter(lambda x: all(value is not None and value == value for value in x.values()))
    
    # Print the number of rows after filtering
    print(f"Rows after filtering: {ds.num_rows}")
    
    # Remove special characters and convert to lowercase
    ds = ds.map(clean_text)

    # Convert to pandas DataFrame
    return ds.to_pandas()

In [6]:
def preprocess_data(ds):
    # Print the number of rows before filtering
    print(f"Rows before filtering: {ds.num_rows}")
    
    # Identify columns with missing values
    missing_values = {col: sum(1 for x in ds[col] if x is None or x != x) for col in ds.column_names}
    print(f"Missing values per column: {missing_values}")
    
    # Create a subset DataFrame with rows that have missing values
    sub_df = ds.filter(lambda x: any(value is None or value != value for value in x.values()))
    
    # Convert the subset to a pandas DataFrame and print the first 5 rows
    sub_df_pd = sub_df.to_pandas()
    print("First 5 rows with missing values:")
    print(sub_df_pd.head())
    
    # Remove rows with any None or NaN values
    ds = ds.filter(lambda x: all(value is not None and value == value for value in x.values()))
    
    # Print the number of rows after filtering
    print(f"Rows after filtering: {ds.num_rows}")
    
    # Remove special characters and convert to lowercase
    ds = ds.map(clean_text)

    # Convert to pandas DataFrame
    return ds.to_pandas()

In [7]:
def preprocess_data(ds):
    # Print the number of rows before filtering
    print(f"Rows before filtering: {ds.num_rows}")
    
    # Identify columns with missing values
    missing_values = {col: sum(1 for x in ds[col] if x is None or x != x) for col in ds.column_names}
    print(f"Missing values per column: {missing_values}")
    
    # Create a subset DataFrame with rows that have missing values
    sub_df = ds.filter(lambda x: any(value is None or value != value for value in x.values()))
    
    # Convert the subset to a pandas DataFrame and print the first 5 rows
    sub_df_pd = sub_df.to_pandas()
    print("First 5 rows with missing values:")
    print(sub_df_pd.head())
    
    # Remove rows with any None or NaN values
    ds = ds.filter(lambda x: all(value is not None and value == value for value in x.values()))
    
    # Print the number of rows after filtering
    print(f"Rows after filtering: {ds.num_rows}")
    
    # Remove special characters and convert to lowercase
    ds = ds.map(clean_text)

    # Convert to pandas DataFrame
    return ds.to_pandas()

In [8]:

# Get the first row to estimate memory usage
first_row = ds['train'][0]
estimated_row_size = sys.getsizeof(first_row)
total_rows = ds['train'].num_rows

# Find available memory to either load the entire dataset or a subset
available_memory = psutil.virtual_memory().available
estimated_memory_required = estimated_row_size * total_rows
available_on_disk_memory = psutil.disk_usage('/').free
MEM_THRESHOLD = 1024**3  # 1 GB, threshold for loading by chunks

print(f"Estimated memory required: {estimated_memory_required} bytes ~ {estimated_memory_required / 1024**3} GB")
print(f"Available RAM: {available_memory} bytes ~ {available_memory / 1024**3} GB")
print(f"Available on-disk memory: {available_on_disk_memory} bytes ~ {available_on_disk_memory / 1024**3} GB")

if estimated_memory_required >= available_on_disk_memory:
    print("Estimated memory required exceeds available on-disk memory. Exiting.")
    sys.exit()

load_by_chunks = False
if estimated_memory_required >= min(available_memory, MEM_THRESHOLD):
    load_by_chunks = True
    print("Estimated memory required exceeds available memory. Loading by chunks...")

Estimated memory required: 1426001288 bytes ~ 1.3280671909451485 GB
Available RAM: 61497024512 bytes ~ 57.27356719970703 GB
Available on-disk memory: 516082073600 bytes ~ 480.6388854980469 GB
Estimated memory required exceeds available memory. Loading by chunks...


In [9]:
# Save data to a CSV file
CHUNK_SIZE = 100000
save_file = 'temp.csv'


print(f'Total rows: {total_rows}')

start_time = time.perf_counter()
if load_by_chunks:
    # Process the dataset in chunks
    for i in range(0, total_rows, CHUNK_SIZE):
        print(f'Processing chunk {i // CHUNK_SIZE + 1} of {total_rows // CHUNK_SIZE + 1}...')
        chunk = ds['train'].select(range(i, min(i + CHUNK_SIZE, total_rows)))
        
        df = preprocess_data(chunk)
        
        # Save the chunk to a CSV file
        if i == 0:
            # Write the header for the first chunk
            df.to_csv(save_file, index=False, mode='w')
        else:
            # Append without writing the header for subsequent chunks
            df.to_csv(save_file, index=False, mode='a', header=False)
else:
    # Process the entire dataset at once
    df = preprocess_data(ds['train'])
    df.to_csv(save_file, index=False)
    
print(f'Finished processing in {time.perf_counter() - start_time} seconds ~ {(time.perf_counter() - start_time) / 60} minutes')

Total rows: 7750007
Processing chunk 1 of 78...
Rows before filtering: 100000
Missing values per column: {'title': 0, 'description': 0, 'summary': 0, 'full_text': 0, '__index_level_0__': 0}


Filter: 100%|██████████| 100000/100000 [00:00<00:00, 104452.98 examples/s]


First 5 rows with missing values:
Empty DataFrame
Columns: [title, description, summary, full_text, __index_level_0__]
Index: []


Filter: 100%|██████████| 100000/100000 [00:00<00:00, 102940.97 examples/s]


Rows after filtering: 100000


...

Filter: 100%|██████████| 100000/100000 [00:00<00:00, 190227.36 examples/s]


First 5 rows with missing values:
Empty DataFrame
Columns: [title, description, summary, full_text, __index_level_0__]
Index: []


Filter: 100%|██████████| 100000/100000 [00:00<00:00, 186137.70 examples/s]


Rows after filtering: 100000


Map: 100%|██████████| 100000/100000 [00:12<00:00, 8310.75 examples/s]
